In [ ]:
  !pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr xgboost -q
  !pip install rouge
  !pip install --upgrade rouge-score

  from transformers import BartForConditionalGeneration, T5ForConditionalGeneration
  from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
  import pandas as pd
  import numpy as np
  import torch
  from sklearn.metrics import mean_squared_error
  from rouge_score import rouge_scorer
  from transformers import PegasusForConditionalGeneration, AutoTokenizer
  from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
  from rouge import Rouge
  from datasets import load_dataset
  from google.colab import drive
  import nltk
  import xgboost as xgb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Initialize the Bart tokenizer and model
bart_tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-cnn')

# Initialize the pegasus tokenizer and model
T5_tokenizer = AutoTokenizer.from_pretrained("t5-large")

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_bart_train= pd.read_csv('/content/drive/MyDrive/GP data/Bart_Train_Summaries(0-11490).csv')
df_t5_train=pd.read_csv('/content/drive/MyDrive/GP data/T5_TRAIN_SUMMARIES 0-11490.csv')
df_bart_test=pd.read_csv('/content/drive/MyDrive/GP data/Bart_Summaries_Test_[0-11490] (1).csv')
df_t5_test=pd.read_csv('/content/drive/MyDrive/GP data/T5_SUMMARIES_[0-11490].csv', encoding='ISO-8859-1')

In [ ]:
import nltk
import string

# download stopwords if not already downloaded
nltk.download('stopwords')
nltk.download('punkt')

# define function to clean text
def clean_text(text):
    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation+ '.'))
    # convert to lowercase
    text = text.lower()
    # tokenize text into words
    words = nltk.word_tokenize(text)
    # remove stopwords
    stopwords = nltk.corpus.stopwords.words('english')
    words = [word for word in words if word not in stopwords]
    # join words back into a string
    text = ' '.join(words)
    return text

In [ ]:
# apply cleaning function to summary column
df_bart_train['summary'] = df_bart_train['summary'].apply(clean_text)

# apply cleaning function to highlights column
df_bart_train['highlights'] = df_bart_train['highlights'].apply(clean_text)

In [ ]:
# apply cleaning function to summary column
df_bart_test['summary'] = df_bart_test['summary'].apply(clean_text)

# apply cleaning function to highlights column
df_bart_test['highlights'] = df_bart_test['highlights'].apply(clean_text)

In [ ]:
# apply cleaning function to summary column
df_t5_test['summary'] = df_t5_test['summary'].apply(clean_text)

# apply cleaning function to highlights column
df_t5_test['highlights'] = df_t5_test['highlights'].apply(clean_text)

In [ ]:
x_t5_train = []
for i, summary in df_t5_train['summary'].items():
    if isinstance(summary, str):
        encoded_summary = T5_tokenizer.encode(summary, truncation=True, max_length=1024, padding='max_length', return_tensors='pt')
        x_t5_train.append((i, encoded_summary.numpy().squeeze()))
    else:
        print(f"Summary at index {i} is not a string: {summary}")

In [ ]:
x_bart_train = []
for i, summary in df_bart_train['summary'].items():
    encoded_summary=bart_tokenizer.encode(summary, truncation=True, max_length=1024, padding='max_length', return_tensors='pt')
    x_bart_train.append((i, encoded_summary.numpy().squeeze()))

In [ ]:
y_train = []
for i, summary in df_bart_train['highlights'].items():
    encoded_summary = bart_tokenizer.encode(summary, truncation=True, max_length=1024, padding='max_length', return_tensors='pt')
    y_train.append((i, encoded_summary.numpy().squeeze()))

In [ ]:
x_t5_test = []
for i, summary in df_t5_test['summary'].items():
    encoded_summary =  bart_tokenizer.encode(summary, truncation=True, max_length=1024, padding='max_length', return_tensors='pt')
    x_t5_test.append((i, encoded_summary.numpy().squeeze()))

In [ ]:
x_bart_test = []
for i, summary in df_bart_test['summary'].items():
    encoded_summary =  bart_tokenizer.encode(summary, truncation=True, max_length=1024, padding='max_length', return_tensors='pt')
    x_bart_test.append((i, encoded_summary.numpy().squeeze()))

In [ ]:
y_test = []
for i, summary in df_bart_test['highlights'].items():
    encoded_summary = bart_tokenizer.encode(summary, truncation=True, max_length=1024, padding='max_length', return_tensors='pt')
    y_test.append((i, encoded_summary.numpy().squeeze()))

In [ ]:
# Convert the input data to numpy arrays
x_t5_train = np.array(x_t5_train)
x_t5_test = np.array(x_t5_test)
x_bart_train = np.array(x_bart_train)
y_train = np.array(y_train)
x_bart_test = np.array(x_bart_test)
y_test = np.array(y_test)

In [ ]:
x_t5_train_indices = np.array([t[0] for t in x_t5_train])
x_t5_train_summaries = np.array([t[1] for t in x_t5_train])
x_bart_train_indices = np.array([t[0] for t in x_bart_train])
x_bart_train_summaries = np.array([t[1] for t in x_bart_train])
y_train_indices = np.array([t[0] for t in y_train])
y_train_summaries = np.array([t[1] for t in y_train])

In [ ]:
x_t5_test_indices = np.array([t[0] for t in x_t5_test])
x_t5_test_summaries = np.array([t[1] for t in x_t5_test])
x_bart_test_indices = np.array([t[0] for t in x_bart_test])
x_bart_test_summaries = np.array([t[1] for t in x_bart_test])
y_test_indices = np.array([t[0] for t in y_test])
y_test_summaries = np.array([t[1] for t in y_test])

In [ ]:
# Stack the BART and Pegasus summaries
predictions_train = np.column_stack((x_bart_train_summaries[0:11490], x_t5_train_summaries))
predictions_test = np.column_stack((x_bart_test_summaries, x_t5_test_summaries))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

In [ ]:
# Train a RandomForestRegressor on BART summaries
rf_base = RandomForestRegressor(n_estimators=1)
rf_base.fit(predictions_train, y_train_summaries[0:11490])

RandomForestRegressor(n_estimators=1)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression

# Train a KNeighborsRegressor on Pegasus summaries
mlp_base = MLPRegressor()
mlp_base.fit(predictions_train, y_train_summaries[0:11490])

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPRegressor()

In [ ]:
# Step 5: Create a new DataFrame to store the predictions
predictions = pd.DataFrame({"bart_preds": rf_base.predict(predictions_test[0:5000]).ravel(), "pegasus_preds": mlp_base.predict(predictions_test[0:5000]).ravel()})

In [ ]:
y_test_flat = y_test_summaries[0:5000].ravel()

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np

# Initialize base estimators
estimators = [
    ('rf',  RandomForestRegressor()),
    ('mlp', MLPRegressor()),
]


# Initialize meta-model with base estimators
meta_model =GradientBoostingRegressor(n_estimators=100)
meta_model.fit(predictions, y_test_flat)

GradientBoostingRegressor()

In [ ]:
test_rf_preds=rf_base.predict(predictions_test[5000:]).ravel()
test_mlp_preds=mlp_base.predict(predictions_test[5000:]).ravel()

In [ ]:
# Step 5: Create a new DataFrame to store the predictions
test_predictions = pd.DataFrame({"bart_preds": test_rf_preds, "pegasus_preds":test_mlp_preds})

In [ ]:
ensemble_preds = meta_model.predict(test_predictions)

In [ ]:
ensemble_preds = ensemble_preds.reshape((6489,-1))

In [ ]:
# set min and max values for token IDs based on tokenizer vocab size
min_value = 0
max_value = bart_tokenizer.vocab_size - 1

# decode summaries from predicted token IDs
ensemble_pred_text = []
for i in range(len(ensemble_preds)):
    summary_ids = ensemble_preds[i]
    # clip token IDs to valid range
    summary_ids = np.clip(summary_ids, min_value, max_value)
    summary_ids = summary_ids.astype(int)
    summary = bart_tokenizer.decode(summary_ids, skip_special_tokens=True, max_length=100)
    ensemble_pred_text.append(summary)

In [ ]:
y_test_text = []
for i in range(len(y_test)):
    encoded_summary = y_test[i][1]
    decoded_summary = bart_tokenizer.decode(encoded_summary.squeeze(), skip_special_tokens=True, max_length=100)
    y_test_text.append( decoded_summary)

In [ ]:
import sys

sys.setrecursionlimit(10000) # set the recursion limit to 10000

rouge = Rouge()
batch_size = 1000
num_batches = (len(ensemble_pred_text) + batch_size - 1) // batch_size # round up to nearest integer

scores = []
for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min((i+1) * batch_size, len(ensemble_pred_text))
    batch_pred_text = ensemble_pred_text[start_idx:end_idx]
    batch_ref_text = y_test_text[start_idx:end_idx]

    batch_scores = []
    for j in range(len(batch_ref_text)):
        reference = str(batch_ref_text[j])
        hypothesis = str(batch_pred_text[j])
        if not reference:
            score = {'rouge-1': {'f': 0, 'p': 0, 'r': 0}, 'rouge-l': {'f': 0, 'p': 0, 'r': 0}}
        else:
            score = rouge.get_scores(hypothesis, reference)
        batch_scores.append(score)

    scores.extend(batch_scores)

In [ ]:
# Get the three ROUGE scores for each summary and calculate the average
rouge_1_scores = [score[0]["rouge-1"]["f"] for score in scores]
rouge_l_scores = [score[0]["rouge-l"]["f"] for score in scores]
average_rouge_1 = sum(rouge_1_scores) / (len(scores))
average_rouge_L = sum(rouge_l_scores) / (len(scores))

In [ ]:
# Save the ROUGE scores to a CSV file
rouge_scores_df = pd.DataFrame({
    "rouge-1": rouge_1_scores,
    "rouge-l": rouge_l_scores,
    "Average-1": average_rouge_1,
    "Average-L": average_rouge_L,
})
rouge_scores_df.to_csv("Bart&T5_Gradient_Rouge_Scores.csv", index=False)